In [2]:
project_name = 'inno_hack'

In [5]:
import numpy as np
import pandas as pd
import sklearn
from importlib import reload
import sys
from os import path

sys.path.append('/home/greg/Personal/hackaton/rolling_drones_1/custom_modules')
import my_sklearn_transformers as trsfm
reload(trsfm)

pd.set_option('display.max_columns', 500)
path_to_data_dir = lambda x: path.join('/home/greg/Personal/hackaton/rolling_drones_1/ml/', *(x if isinstance(x, str) is False else [x]))
path_to_cp_dir = lambda x: path.join('/home/greg/Personal/hackaton/rolling_drones_1/models/', *(x if isinstance(x, str) is False else [x]))
path_to_pipeline_dir = lambda x: path.join('/home/greg/Personal/hackaton/rolling_drones_1/pipelines/', *(x if isinstance(x, str) is False else [x]))

In [6]:
U = pd.read_csv(path_to_data_dir('company_and_its_investor.csv'))

In [7]:
U.head(3)

,Unnamed: 0,inn__investor,name__investor,type__investor,startup_stage__investor,market__investor,services__investor,technologies__investor,type_of_ownership__investor,investment_round__investor,investition_from_dol__investor,investition_to_dol__investor,tech_focus__investor,fund_total_rub__investor,fund_total_dol__investor,num_of_investments__investor,num_of_exits__investor,geography__investor,study_format__investor,num_of_people_in_company_from__investor,num_of_people_in_company_to__investor,num_of_participants__investor,okved_main__investor,okved_secondary__investor,corp_stage__investor,business_model__investor,monetary_support__investor,inn__company,name__company,got_support_from__company,did_get_support__company,service__company,foundation_date__company,tech_focus__company,stage_of_development__company,market__company,technology__company,business_model__company,main_okved__company,okved_secondary__company,msp_category__company,is_export__company,inno_cluster_member__company,skolcovo_member__company,is_inno_company__company,is_startup__company,current_profit__company,current_profit_tax__company,current_revenue__company,type__company
0,0,7703638456,da vinci capital,VentureFund,"['ранний рост', 'расширение']","['cybersecurity', 'transport & logistics', 'bu...",['инвестиции'],"['ar/vr', 'big data', 'блокчейн', 'интернет ве...",частный,"['раунд а', 'раунд в', 'раунд с+']",1000000.0,50000000.0,[],25000.0,397.0,17.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7704794345,"ооо ""геттакси рус""",карта инновационных решений,да,добавлено - карта инновационных решений,2011-11-03,['транспорт и логистика'],расширение,['transport & logistics'],[],['b2c'],62.09,"['82.99', '46.19', '63.11.1', '47.91.2', '69.2...",NaN,нет,да,нет,да,нет,-23.0,-15.0,1507.0,Company
1,1,7728461088,new industry ventures,VentureFund,"['ранний рост', 'расширение', 'посевная']",['cleantech'],['инвестиции'],"['3d моделирование', 'big data', 'интернет вещ...",частный,"['seed', 'раунд а', 'раунд в']",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""скайер ит""",московский акселератор,нет,московский акселератор,2008-12-23,['недвижимость и строительство'],ранний рост,['proptech'],['беспилотники'],['b2b'],72.19,"['62.09', '85.42', '62.02', '63.11.1', '30.30....",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0,Company
2,2,7728461088,new industry ventures,VentureFund,"['ранний рост', 'расширение', 'посевная']",['cleantech'],['инвестиции'],"['3d моделирование', 'big data', 'интернет вещ...",частный,"['seed', 'раунд а', 'раунд в']",500000.0,3000000.0,[],4000.0,62.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7714762021,"ооо ""радуга""",карта инновационных решений,да,добавлено - карта инновационных решений,2008-12-23,['недвижимость и строительство'],ранний рост,['proptech'],['беспилотники'],['b2b'],72.19,"['62.09', '85.42', '62.02', '63.11.1', '30.30....",юл микро,нет,да,да,да,нет,-16450.0,0.0,19127.0,Company


In [8]:
import ast
def try_load_list(val):
    try:
        res = ast.literal_eval(val)
        if isinstance(res, list):
            return res
        else:
            return []
    except Exception as e:
        return []

In [9]:
from sklearn.pipeline import make_union, make_pipeline
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer

onehot_company = ['stage_of_development__company', 'main_okved__company', 'msp_category__company', 'is_export__company', 'inno_cluster_member__company', 'skolcovo_member__company',
                  'is_inno_company__company', 'is_startup__company']
multilabel_company = ['tech_focus__company', 'market__company', 'technology__company', 'business_model__company', 'okved_secondary__company']


num_investor = ['investition_from_dol__investor', 'investition_to_dol__investor', 'fund_total_rub__investor', 
                'fund_total_dol__investor', 'num_of_investments__investor', 'num_of_exits__investor']
onehot_investor = ['type_of_ownership__investor']
multilabel_investor = ['startup_stage__investor', 'market__investor', 'services__investor', 'technologies__investor', 'investment_round__investor', 'tech_focus__investor']

def map_lists(x):
    return np.array([try_load_list(i) for i in x], dtype=object)

preprocessor_X = make_union(
    make_pipeline(
        trsfm.DfSelector(onehot_company),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=''),
        OneHotEncoder(sparse=False, handle_unknown='ignore')
    ),
    *([make_pipeline(
        trsfm.DfSelector([col]),
        FunctionTransformer(np.squeeze),
        FunctionTransformer(map_lists),
        # FunctionTransformer(print),
        trsfm.MultilabelEncoder()
    ) for col in multilabel_company])
)

preprocessor_Y = make_union(
    make_pipeline(
        trsfm.DfSelector(onehot_investor),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=''),
        OneHotEncoder(sparse=False, handle_unknown='ignore')
    ),
    make_pipeline(
        trsfm.DfSelector(num_investor),
        SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0),
        MinMaxScaler()
    ),
    *([make_pipeline(
        trsfm.DfSelector([col]),
        FunctionTransformer(np.squeeze),
        FunctionTransformer(map_lists),
        trsfm.MultilabelEncoder()
    ) for col in multilabel_investor])
)

In [10]:
sklearn.__version__

'0.24.2'

In [11]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1)
for train, test in sss.split(U, U['type_of_ownership__investor'].fillna('')):
    train_idx, test_idx = train, test

In [12]:
X_tr = preprocessor_X.fit_transform(U.iloc[train_idx])
X_te = preprocessor_X.transform(U.iloc[test_idx])
y_tr = preprocessor_Y.fit_transform(U.iloc[train_idx])
y_te = preprocessor_Y.transform(U.iloc[test_idx])

In [14]:
from joblib import dump

dump(preprocessor_X, path_to_pipeline_dir('fund_classifier_preprocessor_X.joblib'))
dump(preprocessor_Y, path_to_pipeline_dir('fund_classifier_preprocessor_Y.joblib'))

['/home/greg/Personal/hackaton/rolling_drones_1/pipelines/fund_classifier_preprocessor_Y.joblib']

In [15]:
unique_fund = U.drop_duplicates(subset=['name__investor'])
investors_space = preprocessor_Y.transform(unique_fund)
investors_space_name = unique_fund['name__investor'].values
fund_name_te = U.iloc[test_idx]['name__investor'].values

In [16]:
investors_space.shape

(59, 77)

In [17]:
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape, y_tr.min(), y_tr.max()

((362, 571), (41, 571), (362, 77), (41, 77), 0.0, 1.0)

# Модель

Генерация предсказания построена так:
Данные о компании, для которой нужно получить рекомендацию, переводяться в n-мерный вектор, который при помощи 2-слойной модели проецируется в m-мерное пространство сервисов. Далее сравнивается L2 норм расстояния от проекции вектора компании до векторов сервисов, и по данным расстояниям сервисы сортируются в порядке убывания релевантности.  

In [18]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.losses import MeanSquaredError

es_cb = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.001, patience=100, verbose=0,
    mode='auto', baseline=None, restore_best_weights=True
)

loss = MeanSquaredError()

l_size = 256
dropo = 0.5

model = tf.keras.Sequential(layers=[
    layers.Dense(l_size, input_shape=(X_tr.shape[1],), activation='sigmoid'),
    layers.Dropout(dropo),
    layers.Dense(y_tr.shape[1], activation='sigmoid')
])

model.compile(optimizer='adam', loss=loss)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               146432    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 77)                19789     
Total params: 166,221
Trainable params: 166,221
Non-trainable params: 0
_________________________________________________________________


2021-11-04 00:43:01.211012: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2021-11-04 00:43:01.225749: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2099940000 Hz
2021-11-04 00:43:01.226050: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5621f64ab330 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-04 00:43:01.226068: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-11-04 00:43:01.227897: I tensorflow/core/common_runtime/process_util.cc:147] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [19]:
model.fit(X_tr, y_tr, validation_data=(X_te, y_te), epochs=3000, callbacks=[es_cb])

Epoch 1/3000
12/12 [==============================] - 0s 35ms/step - loss: 0.2349 - val_loss: 0.1664
Epoch 2/3000
12/12 [==============================] - 0s 16ms/step - loss: 0.1825 - val_loss: 0.1568
Epoch 3/3000
12/12 [==============================] - 0s 8ms/step - loss: 0.1769 - val_loss: 0.1536
Epoch 4/3000
12/12 [==============================] - 0s 13ms/step - loss: 0.1744 - val_loss: 0.1514
Epoch 5/3000
12/12 [==============================] - 0s 12ms/step - loss: 0.1699 - val_loss: 0.1497
Epoch 6/3000
12/12 [==============================] - 0s 13ms/step - loss: 0.1687 - val_loss: 0.1486
Epoch 7/3000
12/12 [==============================] - 0s 13ms/step - loss: 0.1657 - val_loss: 0.1467
Epoch 8/3000
12/12 [==============================] - 0s 14ms/step - loss: 0.1640 - val_loss: 0.1451
Epoch 9/3000
12/12 [==============================] - 0s 13ms/step - loss: 0.1626 - val_loss: 0.1434
Epoch 10/3000
12/12 [==============================] - 0s 15ms/step - loss: 0.1591 - val_los

In [24]:
model.save(path_to_cp_dir('model_fund_classifier.h5'))

In [21]:
tf.__version__

'2.2.0'

In [22]:
from numpy import linalg as LA

preds = model.predict(X_te)
for i in range(len(preds)):
    idx_sorted = np.argsort(np.apply_along_axis(LA.norm, 1, investors_space - preds[i], ord=2))
    print("Pred={}, y={}".format(investors_space_name[idx_sorted[:3]], fund_name_te[i]))

Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['rbf ventures' 'winter capital' 'emery capital'], y=фрии
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['inventure partners' 'flashpoint vc' 'фонд "прообраз"'], y=inventure partners
Pred=['addventure' 'flashpoint vc' 'larix'], y=winter capital
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['inventure partners' 'flashpoint vc' 'фонд "прообраз"'], y=inventure partners
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['фрии' 'the untitled ventures' 'венчурный фонд нти'], y=фонд поддержки проектов нти
Pred=['softline venture partners' 'венчурный фонд мтс' 'yellowrockets'], y=yellowrockets
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=фрии
Pred=['фрии' 'the untitled ventures' 'leta capital'], y=leta capital
Pred=['moscow seed fund' 'rbf ventures' 'dv capital'], y=moscow seed fund
Pred=['фрии' 'the u

In [23]:
from joblib import load

def predict(company, services, answer_in):
    company = company.rename(columns=lambda x: '{}__company'.format(x))
    services = services.rename(columns=lambda x: '{}__investor'.format(x))
    
    X = load(path_to_data_dir('fund_classifier_preprocessor_X.joblib')).transform(np.array([company]))
    services_space = load(path_to_data_dir('fund_classifier_preprocessor_Y.joblib')).transform(services)

    proj = tf.keras.models.load_model('fund_classifier_model.h5').predict(X)[0]

    idx_sorted = np.argsort(np.apply_along_axis(LA.norm, 1, services_space - proj, ord=2))

    return answer_in[idx_sorted]

In [120]:
def train(train_data, val_data):
    pre_X = load(path_to_data_dir('fund_classifier_preprocessor_X.joblib'))
    pre_Y = load(path_to_data_dir('fund_classifier_preprocessor_Y.joblib')).transform(train_data)

    X_tr = pre_X.transform(train_data)
    y_tr = pre_Y.transform(train_data)

    X_te = pre_X.transform(val_data)
    y_te = pre_Y.transform(val_data)

    es_cb = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', min_delta=0.001, patience=10, verbose=0,
        mode='auto', baseline=None, restore_best_weights=True
    )

    model = tf.keras.models.load_model(path_to_data_dir('fund_classifier_model.h5'))
    
    model.fit(X_tr, y_tr, validation_data=(X_te, y_te), epochs=100, callbacks=[es_cb])

    model.save(path_to_data_dir('fund_classifier_model.h5'))

19